# Textual Data in Lily

In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random as rd
import re
import nn
import matplotlib.pyplot as plt
import matplotlib.cm as cm

---
## Data Import

In [59]:
data = pd.read_excel('/Users/desiredewaele/Google Drive/Datasets/lily text.xlsx', index_col=0)
data.tail(3)

,Panellid,E-mailadres,Achternaam,Voornaam,Taal,Geslacht,Geboortedatum,Veld 1,Veld 5,Straat,...,6.4. onze website,7.1. Naamsbekendheid,7.2. Prijs,7.3. Contact met de klantendienst,7.4. Groene energie,"7.5. Advies vrienden, familie, collega's",7.6. Duidelijkheid en correctheid facturen,7.7. Other,"7.7. Ander, gelieve te specificeren",prijskwaliteit
Respondent,,,,,,,,,,,,,,,,,,,,,
10398,35855057,gaston@f-a-q.be,Gaston Verelst,NaN,nl,Man,NaN,150153215.0,NaN,Groenstraat,...,NVT,Niet belangrijk,Uiterst belangrijk,Belangrijk,Niet belangrijk,Niet belangrijk,Uiterst belangrijk,NaN,NaN,8.0
10399,35855871,familiewilms@hotmail.com,Wilhelmus Wilms,NaN,nl,Man,NaN,150346421.0,NaN,Koekoeklaan,...,NVT,Niet belangrijk,Zeer belangrijk,Niet belangrijk,Niet belangrijk,NaN,Belangrijk,NaN,NaN,4.0
10400,35855594,doke.brouns@telenet.be,Doke Brouns,NaN,nl,Vrouw,NaN,150136365.0,NaN,Steenberg,...,6,Niet belangrijk,Uiterst belangrijk,Belangrijk,Belangrijk,Belangrijk,Zeer belangrijk,NaN,NaN,3.0


---
## Data Preproccessing

---
### Feature Selection

In [60]:
data = data.iloc[:,31:35]
data.columns = ['label', 'text1', 'text2', 'text3']

In [61]:
data.tail()

,label,text1,text2,text3
Respondent,,,,
10396,7,NaN,NaN,NaN
10397,8,NaN,NaN,lagere prijzen ???
10398,6,Ik denk niet dat Essent beter of slechter is d...,NaN,NaN
10399,5,rekening even hoog als bij vorige leverancier,NaN,NaN
10400,5,Duur,NaN,NaN


---
### Feature Formatting

In [62]:
data.text1 = data.text1.apply(lambda x: x.encode('utf8') if isinstance(x, unicode) else ' ')
data.text2 = data.text2.apply(lambda x: x.encode('utf8') if isinstance(x, unicode) else ' ')
data.text3 = data.text3.apply(lambda x: x.encode('utf8') if isinstance(x, unicode) else ' ')

In [63]:
data['text'] = data.text1.str.cat(data.text2).str.cat(data.text3)
data = data[['label', 'text']]

In [64]:
def textconverter(text):
    text = text.lower()                      # Change text to lowercase
    text = re.sub('[^(a-z)]| ', ' ', text)   # Change all not-text to spaces
    text = re.sub(' +',' ',text)             # Remove all redundant spaces
    text = text.strip()                      # Remove outer whitespace
    return text

In [65]:
data.text = data.text.apply(lambda x: textconverter(x))
data.tail()

,label,text
Respondent,,
10396,7,
10397,8,lagere prijzen
10398,6,ik denk niet dat essent beter of slechter is d...
10399,5,rekening even hoog als bij vorige leverancier
10400,5,duur


---
### Missing Values

In [66]:
data = data[data.text != '']
data.tail()

,label,text
Respondent,,
10395,5,er
10397,8,lagere prijzen
10398,6,ik denk niet dat essent beter of slechter is d...
10399,5,rekening even hoog als bij vorige leverancier
10400,5,duur


In [67]:
len(data)

6523

---
### Label Formatting

In [68]:
print(len(data[data.label > 7]))
print(len(data[data.label <= 7]))

3248
3275


In [69]:
data.label = data.label.apply(lambda x: 1 if x > 7 else 0)
data.tail()

,label,text
Respondent,,
10395,0,er
10397,1,lagere prijzen
10398,0,ik denk niet dat essent beter of slechter is d...
10399,0,rekening even hoog als bij vorige leverancier
10400,0,duur


---
## Save Tidy Data

In [85]:
data.to_csv('/Users/desiredewaele/Google Drive/Datasets/lilyTidyText.xlsx')

---
## Get Word Counts & Tfidfs with Sklearn

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

a = CountVectorizer()
b = CountVectorizer(ngram_range=(1, 2))
c = CountVectorizer(ngram_range=(1, 3))
d = TfidfVectorizer(sublinear_tf=True)
e = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3))
f = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3))

In [71]:
onegrams = pd.DataFrame(a.fit_transform(data.text).toarray(), columns=a.get_feature_names())
twograms = pd.DataFrame(b.fit_transform(data.text).toarray(), columns=b.get_feature_names())
trigrams = pd.DataFrame(c.fit_transform(data.text).toarray(), columns=c.get_feature_names())
onetfidf = pd.DataFrame(d.fit_transform(data.text).toarray(), columns=d.get_feature_names())
twotfidf = pd.DataFrame(e.fit_transform(data.text).toarray(), columns=e.get_feature_names())
tritfidf = pd.DataFrame(f.fit_transform(data.text).toarray(), columns=f.get_feature_names())

In [72]:
tritfidf.tail()

,aagesloten,aagesloten via,aagesloten via groepsaankoop,aan,aan als,aan als de,aan andere,aan andere dat,aan anderen,aan anderen hoeft,...,zwart op,zwart op wit,zweem,zweem van,zweem van onbereikbaarheid,zyn,zyn ben,zyn ben aant,zyn voor,zyn voor nieuwe
6518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
def getTopValues(s):
    tmp = s.sort_values(ascending=False)[:3]
    return tuple(tmp.index) #dict(zip(tmp.index, tmp))

In [74]:
onegrams.apply(getTopValues, axis=1).tail()

6518       (er, zyn, fiabilit)
6519    (prijzen, lagere, zyn)
6520        (essent, niet, de)
6521    (even, hoog, rekening)
6522     (duur, zyn, fiabilit)
dtype: object

---
## Logistic Regression

In [75]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(tritfidf.as_matrix(), data.label, train_size=0.7)

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

lr = LogisticRegression()         # Logistic Regression
nb = GaussianNB()                 # Gaussian Naive Bayes
sd = SGDClassifier()              # Stochastic Gradient Descent
dt = DecisionTreeClassifier()     # Decision Tree
ft = RandomForestClassifier()     # Random Forrest Tree
et = ExtraTreesClassifier()       # Extra Trees

In [77]:
lr.fit(trainX, trainY)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [78]:
print(lr.score(validX, validY))
print(lr.score(trainX, trainY))

0.706693919264
0.922251423565


In [79]:
predictions = data.loc[validY[:10].index]
predictions['prediction'] = lr.predict(validX[:10])
predictions

,label,text,prediction
Respondent,,,
2042,0,geen zoomit betalingen mogelijk,0
5941,1,prix competitifs,1
8625,1,systheme de facturation,0
1058,1,niet alleen actie s doen naar nieuwe klanten t...,0
6107,0,vous avez deja vu le temps que lon perd chez v...,0
6411,1,tr s bon service rien redire,1
3981,1,als goedkoopste voor mij bij groepsaankoop,1
3636,0,verwarrende communicatie,0
2041,0,geen reden waarom wel essent spring er niet uit,0


---
## TensorFlow

In [80]:
import collections
import random
import math
from sklearn.manifold import TSNE
from matplotlib import pylab

In [81]:
text = sum(list(data.text.apply(lambda x: x.split())), [])
print(len(data))
print(len(set(data)))

6523
2


In [84]:
VOCAB = 5000
EMBED = 128

embeddings = tf.Variable(tf.random_uniform([VOCAB, EMBED], -1.0, 1.0))
w = tf.Variable(tf.truncated_normal([VOCAB, EMBED], stddev=1.0 / math.sqrt(EMBED)))
b = tf.Variable(tf.zeros([VOCAB]))

In [55]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(VOCAB - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(text)
print('Most common words:', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

('Most common words:', [['UNK', 1], (1, 3775), (0, 3188), (2, 1346), (3, 1331)])
('Sample data', [2, 71, 1918, 910, 483, 2, 183, 123, 287, 57], [0, 70, 1917, 909, 482, 0, 182, 122, 286, 56])


In [57]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for SKIPS, WINDOW in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=SKIPS, skip_window=WINDOW)
    print('\nwith num_skips = %d and skip_window = %d:' % (SKIPS, WINDOW))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

('data:', [0, 70, 1917, 909, 482, 0, 182, 122])

with num_skips = 2 and skip_window = 1:
('    batch:', [70, 70, 1917, 1917, 909, 909, 482, 482])
('    labels:', [1917, 0, 909, 70, 482, 1917, 909, 0])

with num_skips = 4 and skip_window = 2:
('    batch:', [1917, 1917, 1917, 1917, 909, 909, 909, 909])
('    labels:', [0, 70, 482, 909, 70, 0, 1917, 482])


In [ ]:
BATCH = 128
EMBED = 128 # Dimension of the embedding vector.
WINDOW = 1 # How many words to consider left and right.
SKIPS = 2 # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the validation samples 
# to the words that have a low numeric ID, which by construction are also the most frequent. 
valid_size = 8 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    tfTrainX = tf.placeholder(tf.int32, shape=[BATCH])
    tfTrainY = tf.placeholder(tf.int32, shape=[BATCH, 1])
    tfValidX = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, EMBED], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal(
        [vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(EMBED)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(
        weights=softmax_weights, biases=softmax_biases, inputs=embed, 
        labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings = normalized_embeddings.eval()

In [ ]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [ ]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15,15))  # in inches
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
    pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

In [ ]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

In [ ]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        
        # Look up embeddings for inputs.
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                         biases=nce_biases,
                                         labels=train_labels,
                                         inputs=embed,
                                         num_sampled=num_sampled,
                                         num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [ ]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], {train_inputs: batch_inputs, train_labels: batch_labels})
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

---
## Neural Network

---
### Visualizer Functions
These are some helper functions

In [ ]:
def plotter(history):
    at, av, lt, lv = zip(*history)
    fig = plt.figure(figsize=(15, 8)); ax1 = fig.add_subplot(221); ax2 = fig.add_subplot(222)

    ax1.plot(np.arange(0, len(at), 1), at,".-", color='#2A6EA6', label="Training: {0:.2f}%".format(at[-1]))
    ax1.plot(np.arange(0, len(av), 1), av,".-", color='#FFA933', label="Validation: {0:.2f}%".format(av[-1]))
    ax1.grid(True); ax1.legend(loc="lower right"); ax1.set_title("Accuracy per epoch")

    ax2.plot(np.arange(0, len(lt), 1), lt,".-", color='#2A6EA6', label="Training: {0:.2f}".format(lt[-1]))
    ax2.plot(np.arange(0, len(lv), 1), lv,".-", color='#FFA933', label="Validation: {0:.2f}".format(lv[-1]))
    ax2.grid(True); ax2.legend(loc="upper right"); ax2.set_title("Cost per epoch")
    plt.show()

---
### Data Preproccessing
Reformat into a TensorFlow-friendly shape:
- all input should be numpy
- labels as float 1-hot encodings.

In [ ]:
print('Training set:', twotfidf.shape, dataNL.label.shape)

In [ ]:
data = twotfidf.values
labels = pd.get_dummies(dataNL.label).values

In [ ]:
print('Training set:', data.shape, labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(data, labels, train_size=0.7)

In [ ]:
print('Training set:', trainX.shape, trainY.shape)
print('Validation set:', validX.shape, validY.shape)

In [ ]:
SIZE = trainX.shape[1]
LABELS = 2
BREAKS = 5
BATCH = 64
HIDDEN1 = 10
RATE = 0.01
STEPS = 100

In [ ]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tfDataX = tf.placeholder(tf.float32, shape=(None, SIZE))
    tfDataY = tf.placeholder(tf.float32, shape=(None, LABELS))

    # Variables.
    w1 = tf.Variable(tf.truncated_normal([SIZE, HIDDEN1], stddev=np.sqrt(2.0/SIZE)))
    w2 = tf.Variable(tf.truncated_normal([HIDDEN1, LABELS], stddev=np.sqrt(2.0/SIZE)))
    b1 = tf.Variable(tf.zeros([HIDDEN1]))
    b2 = tf.Variable(tf.zeros([LABELS]))

    # Model.
    def model(x, training=False):
        x = tf.nn.relu(tf.matmul(x, w1) + b1)
        return tf.matmul(x, w2) + b2
  
    # Training computation.
    logits = model(tfDataX, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tfDataY))
    rate = tf.train.exponential_decay(0.5, tf.Variable(0), 4000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(RATE).minimize(loss)

    # Predictions and Accuracy.
    predictions = {"classes": tf.argmax(model(tfDataX), axis=1),"probabilities": tf.nn.softmax(model(tfDataX))}
    accuracy = tf.reduce_mean(tf.to_float(tf.equal(predictions["classes"], tf.argmax(tfDataY, axis=1)))) * 100

In [ ]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    history = []
    for step in range(STEPS):
        offset = (step * BATCH) % (trainY.shape[0] - BATCH)
        batchX = trainX[offset:(offset + BATCH), :]
        batchY = trainY[offset:(offset + BATCH), :]
        session.run(optimizer, {tfDataX: trainX, tfDataY: trainY})
        if(step % (STEPS // BREAKS) == 0):
            lt, at = session.run([loss, accuracy], {tfDataX: trainX, tfDataY: trainY})
            lv, av = session.run([loss, accuracy], {tfDataX: validX, tfDataY: validY})
            history.append((at, av, lt, lv))
            print ".",
    predictions = session.run(predictions, {tfDataX: validX})
    #accuracy = session.run(accuracy, {tfDataX: testX, tfDataY: testY})
    #print('\nTest accuracy: %.2f%%' % accuracy)

In [ ]:
plotter(history)